# Merge a dataset through 2020 with audio feature metadata (importantly the genre), pull lyrics from dataset with pre-pulled lyrics through 2019, snag 2020 lyrics via genius API

In [1]:
import pandas as pd
import numpy as np

In [2]:
np.__version__

'1.19.5'

## 1. Read in the list of songs

In [3]:
songs_df = pd.read_csv('./data/1 DONE RIGHT OUTPUT unique songs.csv')

In [4]:
songs_df

,song_id,chart_position,chart_debut,song,performer
0,#9 DreamJohn Lennon,9,1974-12-21,#9 Dream,John Lennon
1,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,4,2002-10-26,'03 Bonnie & Clyde,Jay-Z Featuring Beyonce Knowles
2,'65 Love AffairPaul Davis,6,1982-02-27,'65 Love Affair,Paul Davis
3,('til) I Kissed YouThe Everly Brothers,4,1959-08-15,('til) I Kissed You,The Everly Brothers
4,(Can't Live Without Your) Love And AffectionNe...,1,1990-07-07,(Can't Live Without Your) Love And Affection,Nelson
...,...,...,...,...,...
5061,everything i wantedBillie Eilish,8,2019-11-23,everything i wanted,Billie Eilish
5062,iSpyKYLE Featuring Lil Yachty,4,2017-01-14,iSpy,KYLE Featuring Lil Yachty
5063,interludeJ. Cole,8,2021-05-22,interlude,J. Cole
5064,"my.lifeJ. Cole, 21 Savage & Morray",2,2021-05-29,my.life,"J. Cole, 21 Savage & Morray"


## 2. Read in any already fetched lyrics

In [5]:
existing_lyrics_df = pd.read_csv('./data/Lyrics from Lyric Genius API with non-matches left null.csv')

In [6]:
existing_lyrics_df

,song_id,lyrics
0,#9 DreamJohn Lennon,#9 Dream Lyrics[Verse 1]\nSo long ago\nWas it ...
1,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,"’03 Bonnie & Clyde Lyrics[Intro: JAY-Z]\nUhh, ..."
2,'65 Love AffairPaul Davis,’65 Love Affair LyricsI was a car hop\nYou wer...
3,('til) I Kissed YouThe Everly Brothers,(’Til) I Kissed You Lyrics[Verse]\nNever felt ...
4,(Can't Live Without Your) Love And AffectionNe...,(Can’t Live Without Your) Love And Affection L...
...,...,...
5061,everything i wantedBillie Eilish,​everything i wanted Lyrics[Verse 1]\nI had a ...
5062,iSpyKYLE Featuring Lil Yachty,"​iSpy Lyrics[Intro: KYLE & Lil Yachty]\nMan, f..."
5063,interludeJ. Cole,Once an Addict (Interlude) Lyrics[Intro]\nRigh...
5064,"my.lifeJ. Cole, 21 Savage & Morray",October 2021 Singles Release Calendar Lyrics10...


## 3. Determine if there are any songs we haven't tried fetching lyrics for

In [7]:
songs_we_have_not_tried_fetching_lyrics_for = songs_df[
    ~songs_df['song_id'].isin(existing_lyrics_df['song_id'])
]
songs_we_have_not_tried_fetching_lyrics_for

,song_id,chart_position,chart_debut,song,performer


# Scrape Lyrics for 2020 from genius.com API

### Setup

In [8]:
import lyricsgenius

In [9]:
GENIUS_CLIEN_API_KEY = '8ZiCNVQqbSF6NM0nwVnVv6Y0EBR7BsVQ9HaVgPdqJOZR1l6XlymR-pUUi6z3GHPx' # be nice :)
genius = lyricsgenius.Genius(GENIUS_CLIEN_API_KEY)

In [10]:
def get_lyrics(title='', artist=''):
    try:
        return genius.search_song(title, artist).lyrics
    except:
        print('not found: ${title}')

## Do the actual Genius API calls, stash results to disk

In [11]:
# get_lyrics(title='Adore You', artist='Harry Styles')
fetched_lyrics = songs_we_have_not_tried_fetching_lyrics_for.apply(
    lambda row: get_lyrics(title=row['song'], artist=row['performer']),
    axis=1
)

### TODO: ADD THE NEW FETCHED SONGS TO EXISTING DATA & SAVE TO DISK

In [12]:
# lyrics_to_save_to_disk_df = songs_df[['song_id']].copy()
# lyrics_to_save_to_disk_df['lyrics'] = fetched_lyrics
# lyrics_to_save_to_disk_df.to_csv('./data/Lyrics from Lyric Genius API with non-matches left null.csv', index=False)

## Done, export songs who have lyrics

In [22]:
ready_for_export = pd.merge(
    left=songs_df, 
    left_on='song_id', 
    right=existing_lyrics_df, 
    right_on='song_id'
).dropna(axis='rows', subset=['lyrics'])

ready_for_export

,song_id,chart_position,chart_debut,song,performer,lyrics
0,#9 DreamJohn Lennon,9,1974-12-21,#9 Dream,John Lennon,#9 Dream Lyrics[Verse 1]\nSo long ago\nWas it ...
1,'03 Bonnie & ClydeJay-Z Featuring Beyonce Knowles,4,2002-10-26,'03 Bonnie & Clyde,Jay-Z Featuring Beyonce Knowles,"’03 Bonnie & Clyde Lyrics[Intro: JAY-Z]\nUhh, ..."
2,'65 Love AffairPaul Davis,6,1982-02-27,'65 Love Affair,Paul Davis,’65 Love Affair LyricsI was a car hop\nYou wer...
3,('til) I Kissed YouThe Everly Brothers,4,1959-08-15,('til) I Kissed You,The Everly Brothers,(’Til) I Kissed You Lyrics[Verse]\nNever felt ...
4,(Can't Live Without Your) Love And AffectionNe...,1,1990-07-07,(Can't Live Without Your) Love And Affection,Nelson,(Can’t Live Without Your) Love And Affection L...
...,...,...,...,...,...,...
5060,amariJ. Cole,5,2021-05-29,amari,J. Cole,"4:44 Lyrics[Paroles de ""4:44"" par. ""Apollonthe..."
5061,everything i wantedBillie Eilish,8,2019-11-23,everything i wanted,Billie Eilish,​everything i wanted Lyrics[Verse 1]\nI had a ...
5062,iSpyKYLE Featuring Lil Yachty,4,2017-01-14,iSpy,KYLE Featuring Lil Yachty,"​iSpy Lyrics[Intro: KYLE & Lil Yachty]\nMan, f..."
5063,interludeJ. Cole,8,2021-05-22,interlude,J. Cole,Once an Addict (Interlude) Lyrics[Intro]\nRigh...


In [23]:
ready_for_export.to_csv('./data/2 DONE RIGHT OUTPUT Songs with lyrics.csv', index=False)